In [1]:
!pip install cassandra-driver

     |████████████████████████████████| 3.8 MB 5.6 MB/s 


In [3]:
import cassandra
print (cassandra.__version__)

3.25.0


In [4]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config= {
        'secure_connect_bundle': '/content/secure-connect-database-2109.zip'
}
auth_provider = PlainTextAuthProvider('QbQNLWKUKjrtyXGCdEEFxLEe', '1Y02.CofOkhlpmE_r2-ljIz,LaZZ..wnFK,Lc85Q,quN913nq1PQc-1q1C7zoMIyOIPSrcLx2tvw.K9pv.rZ4STsv+8QaNfbITRUdTwZaQaLmgXs9x.BJHdpw8amaNk6')
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
    print(row[0])
else:
    print("An error occurred.")

4.0.0.6816


In [5]:
session.execute("USE mykeyspace")

In [23]:
# Extraer las filas de una tabla
filas = session.execute("SELECT * FROM tabla1;")

# Mostrar el contenido de las filas
for fila in filas:
    #print(fila)
    print(fila.home_id, fila.datetime)

H01474777 2014-05-23 18:28:41
H01474777 2014-05-23 07:44:23
H01474777 2014-05-22 17:22:15
H01474777 2014-05-22 11:44:07
H01474777 2014-05-22 11:25:00
H01474777 2014-05-22 11:23:59
H01474777 2014-05-22 07:44:13
H01474777 2014-05-21 18:30:33
H01474777 2014-05-21 07:32:16
H01033638 2014-05-22 07:45:28
H01033638 2014-05-21 19:01:46
H01033638 2014-05-21 16:58:39
H01033638 2014-05-21 13:02:11
H01033638 2014-05-21 07:55:58
H01033638 2014-05-21 07:50:43
H01033638 2014-05-21 07:50:22
H04177 2014-05-21 07:32:16
H02257222 2014-05-23 18:06:58
H02257222 2014-05-23 07:49:36
H02257222 2014-05-22 21:59:44
H02257222 2014-05-21 05:29:47
H01545551 2014-05-23 18:14:53
H01545551 2014-05-23 08:28:16
H01545551 2014-05-22 18:35:29
H01545551 2014-05-22 08:32:22
H01545551 2014-05-21 18:41:02
H01545551 2014-05-21 08:30:14
H00999943 2014-05-23 18:56:23
H00999943 2014-05-23 08:52:19
H00999943 2014-05-22 19:10:56
H00999943 2014-05-22 08:55:10
H00999943 2014-05-21 19:03:33
H00999943 2014-05-21 09:05:54


In [24]:
filas2 = session.execute("SELECT home_id, code_used FROM tabla1 WHERE home_id='H01474777';")

for fila in filas2:
    print(fila)

Row(home_id='H01474777', code_used='5599')
Row(home_id='H01474777', code_used='5599')
Row(home_id='H01474777', code_used='5599')
Row(home_id='H01474777', code_used=None)
Row(home_id='H01474777', code_used=None)
Row(home_id='H01474777', code_used=None)
Row(home_id='H01474777', code_used='5599')
Row(home_id='H01474777', code_used='5599')
Row(home_id='H01474777', code_used='5599')


## Parte 2

In [ ]:
# Creación de un Keyspace (no se puede correr en AstraDB)
session.execute("""
CREATE KEYSPACE IF NOT EXISTS my_keyspace4
WITH REPLICATION =
{'class': 'SimpleStrategy', 'replication_factor': 1}
""")

In [12]:
# Creación de una tabla

creacion_tabla = """
CREATE TABLE IF NOT EXISTS biblioteca
(anho int, artista text, album text, PRIMARY KEY(anho,artista))
"""

session.execute(creacion_tabla)

In [13]:
def str_tabla(nombre, header, primary_key):
    cadena = "CREATE TABLE IF NOT EXISTS " + nombre + " (" + header + ', PRIMARY_KEY(' + primary_key + '))'
    #session.execute(cadena)
    print(cadena)

header = "anho int, artista text, album text"
pkey = "anho,artista"
str_tabla("biblioteca2", header, pkey)

CREATE TABLE IF NOT EXISTS biblioteca2 (anho int, artista text, album text, PRIMARY_KEY(anho,artista))


In [14]:
# Cadena de caracteres para facilitar el ingreso de datos a la tabla
# Los valores %s se reemplazan en ()
insertar = "INSERT INTO biblioteca (anho, artista, album) VALUES (%s,%s,%s)"

In [15]:
# Añadir elementos a la tabla biblioteca
session.execute(insertar, (1980, "Artista1", "album1"))
session.execute(insertar, (1990, "Artista2", "albumA"))
session.execute(insertar, (2000, "Artista2", "albumB"))

In [17]:
# Insertar datos de un archivo csv
# Contenido de musica.csv:
#      1995,"Artista1","AlbumC"
#      2010,"Artista5","AlbumD"

import csv
f = open('musica.csv', 'r')
lineas = csv.reader(f, delimiter=',')

for linea in lineas:
    session.execute(insertar, (int(linea[0]), linea[1], linea[2]))

In [18]:
# Se puede verificar usando select *
filas = session.execute("SELECT * FROM biblioteca")
for fila in filas:
    print(fila)

Row(anho=1995, artista='Artista1', album='AlbumC')
Row(anho=2010, artista='Artista5', album='AlbumD')
Row(anho=1990, artista='Artista2', album='albumA')
Row(anho=2000, artista='Artista2', album='albumB')
Row(anho=1980, artista='Artista1', album='album1')
